In [17]:
# !pip install cellrank
# !pip install scanpy
# !pip install scipy
# !pip install anndata

# rest this dataset for now. 

In [1]:
%%bash

source downloadData.sh

# !mkdir -p ./data
# minimal TF combinations for inducing granulosa-like cells
download_dataset "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE213156&format=file&file=GSE213156%5FTF%5Fgene%5Fexpression%2Ecsv%2Egz" "./data"  "GSE213156_TF_gene_expression.csv.gz"
gunzip_download "./data/GSE213156_TF_gene_expression.csv.gz"

The file ./data/GSE213156_TF_gene_expression.csv.gz already exists. Skipping download.
File ./data/GSE213156_TF_gene_expression.csv already exists. Skipping.


In [2]:
import cellrank as  cr
import scanpy as sc
from scipy import sparse
import pandas as pd
import numpy as np
from anndata import AnnData

 Transcript abundance (TPM) measurements for induced pluripotent stem cells (iPSCs), COV434 ovarian cancer cells, and iPSC-derived granulosa-like cells. 2. Differential gene expression analysis to identify effects of transcription factor overexpression during iPSC differentiation.

In [1]:

# Load h5ad file
df = pd.read_csv('data/GSE213156_TF_gene_expression.csv')

['Gene name', 'Gene stable ID version', 'baseMean', 'log2FoldChange_WT1',
'padj_WT1', 'log2FoldChange_GATA4', 'padj_GATA4',
'log2FoldChange_NR5A1', 'padj_NR5A1', 'log2FoldChange_RUNX1',
'padj_RUNX1', 'log2FoldChange_FOXL2', 'padj_FOXL2',
'log2FoldChange_TCF21', 'padj_TCF21', 'log2FoldChange_COV434',
'padj_COV434', 'log2FoldChange_FOXL2_plus', 'padj_FOXL2_plus',
'log2FoldChange_NOTFS', 'padj_NOTFS']

# the nine TFs are:
    # WT1
    # GATA4
    # NR5A1
    # RUNX1
    # FOXL2
    # TCF21
    # COV434
    # FOXL2_plus
    # NOTFS

# though according to the paper there should be like 20 odd 

# Create anndata object
adata = AnnData(X=df.values, obs=pd.DataFrame(index=df.index), var=pd.DataFrame(index=df.columns))

print(adata.obs.head())
print(adata.var.head())

# this data is just the abundance of the log2fold change of the TFs. Does not have transcription information? 


NameError: name 'pd' is not defined

Here we make both of the anndata objects sparse and change some of the obsm keys. Then we save the new anndata object to h5ad files.